## 配置环境

In [2]:
# 下载模型
!export HF_ENDPOINT=https://hf-mirror.com && huggingface-cli download --resume-download --local-dir-use-symlinks False IDEA-CCNL/Ziya2-13B-Chat --local-dir IDEA-CCNL/Ziya2-13B-Chat

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 1 files:   0%|                                   | 0/1 [00:00<?, ?it/s]downloading https://hf-mirror.com/IDEA-CCNL/Ziya2-13B-Chat/resolve/e5d697b2daf4ec1f5f12036bef2597245d826910/pytorch_model-00001-of-00003.bin to /root/.cache/huggingface/hub/models--IDEA-CCNL--Ziya2-13B-Chat/blobs/f712a4eb70a485dab8dbaf82a74271e0ded2bb14976078f55d7bba988e349875.incomplete

pytorch_model-00001-of-00003.bin:  88%|██████████▌ | 8.76G/9.90G [00:00<?, ?B/s]
pytorch_model-00001-of-00003.bin:  89%|███▌| 8.77G/9.90G [00:01<03:29, 5.41MB/s]
pytorch_model-00001-of-00003.bin:  89%|███▌| 8.78G/9.90G [00:02<02:05, 8.96MB/s]
pytorch_model-00001-of-00003.bin:  89%|███▌| 8.79G/9.90G [00:03<01:32, 12.0MB/s]
pytorch_model-00001-of-00003.bin:  89%|███▌| 8.80G/9.90G [00:03<01:19, 13.8MB/s]
pytorch_model-00001-of-00003.bin:  89%|███▌| 8.81G/9.

## 官方Demo
- [官方Demo🔗](https://huggingface.co/IDEA-CCNL/Ziya2-13B-Chat)

In [3]:
from transformers import AutoTokenizer,AutoModel
import torch

device = torch.device("cuda")
model_path="/root/IDEA-CCNL/Ziya2-13B-Chat"
model = AutoModel.from_pretrained(model_path,torch_dtype=torch.float16).to(device) # 官方版本是 AutoModelForCausalLM bfloat16, float16精度更低
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

Loading checkpoint shards: 100%|██████████| 3/3 [00:18<00:00,  6.09s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 136.00 MiB. GPU 0 has a total capacty of 23.65 GiB of which 54.81 MiB is free. Process 136864 has 23.59 GiB memory in use. Of the allocated memory 23.21 GiB is allocated by PyTorch, and 1.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
messages = [{"role": "user", "content": "手机如果贴膜贴了一张防指纹的钢化膜，那屏幕指纹解锁还有效吗？"}]
user_prefix = "<human>:"
assistant_prefix = "<bot>:"
separator = "\n"
prompt = []
for item in messages:
    prefix = user_prefix if item["role"] == "user" else assistant_prefix
    prompt.append(f"{prefix}{item['content']}")
prompt.append(assistant_prefix)
prompt = separator.join(prompt)


input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generate_ids = model.generate(
            input_ids,
            max_new_tokens=512, 
            do_sample = True, 
            top_p = 0.9, 
            temperature = 0.85, 
            repetition_penalty=1.05, 
            eos_token_id=tokenizer.encode("</s>"), 
            )
output = tokenizer.batch_decode(generate_ids)[0]
print(output)